## Transformer 구조 구현

In [1]:
pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


#### 1. 데이터 확인

In [2]:
# data를 저장할 directory 확인
data_dir = "./data"

#### 2. Imports

In [5]:
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import sentencepiece as spm

import torch
import torch.nn as nn
import torch.nn.functional as F

#### 3. 폴더의 목록을 확인
data_dir 목록을 확인 합니다.

In [4]:
for f in os.listdir(data_dir):
  print(f)

.DS_Store
kor_w2v_cbow
kor_w2v_cbow.model
kor_w2v_skipgram
kor_w2v_skipgram.model
kowiki.model
kowiki.txt
kowiki.vocab
naver.model
naver.vocab
naver_review.txt
ratings_train.csv
ratings_train.txt


#### 4. Vocab 및 입력
Sentencepiece를 활용해 미리 만든 voca를 로드함  
: wiki corpus로 만들어 놓음

로딩된 vocab을 이용해 input을 만듭니다.

In [22]:
# vocab 만들기
# 입력값 구성 -> 입력값에 대해서 인코딩 [인덱스 변환]-> 입력값 출력
vocab_file = f"{data_dir}/kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

# 입력 테스트
lines = [
    "겨울은 추워요.",
    "감기 조심하세요."
]

# input
inputs = []
for line in lines:
    pieces = vocab.encode_as_pieces(line) # 해당 라인을 토큰으로 바꿈 (EX:_나는)
    ids = vocab.encode_as_ids(line) # input은 index로 바꿈
    inputs.append(torch.tensor(ids))
    print(pieces)
    print(ids)


# 입력 길이를 맞춰주기 위해 padding 수행: input의 최대 길이에 맞춰서 패딩 수행
inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)\

# shape
print(inputs.size()) # 2*8 행렬 생성 / 배치사이즈 * input 최대 맥스 length(패딩포함)

# 값
print(inputs)



['▁겨울', '은', '▁추', '워', '요', '.']
[3234, 3744, 205, 4081, 3902, 3730]
['▁감', '기', '▁조', '심', '하', '세', '요', '.']
[199, 3746, 54, 3974, 3736, 3826, 3902, 3730]
torch.Size([2, 8])
tensor([[3234, 3744,  205, 4081, 3902, 3730,    0,    0],
        [ 199, 3746,   54, 3974, 3736, 3826, 3902, 3730]])


#### 5. Embedding

#### - Input Embedding

In [24]:
# input 값에 대한 embedding 수행 = 1) 일반적인 인풋에 대한 임베딩, 2) 포지션에 관한 임베딩
# 일반적인 인풋에 대한 임베딩 
n_vocab = len(vocab)
print(n_vocab)
d_hidn = 128 # 보통의 hidden size: 512
nn_emb = nn.Embedding(n_vocab, d_hidn)# 임베딩 레이어를 초기화할 떄, 외부 임베딩을 사용하지 않고 입력된 텍스트에 대해 학습을 함
input_embs= nn_emb(inputs)
print(input_embs.size())

# 임베딩 벡터는 어떻게 구현이 될까? 
# 임베딩 벡터는 구현이 되면 어떻게 쓰냐 -> voca가 있으면 처리를 함
# 처음의 차원 보카의 사이즈(8007) -> 히든 사이즈 128
# torch.Size([2, 8, 128]) 배치사이즈, input의 최대 length, 히든사이즈


8007
torch.Size([2, 8, 128])


##### - Position Embedding

1. 문장의 position 별 angle 값을 구함  
2. 구해진 angle 중 짝수 index의 값에 대한 sin 값을 구합니다.  
3. 구해진 angle 중 홀수 index의 값에 대한 cos 값을 구합니다.

In [25]:
""" sinusoid position embedding """
def get_sinusoid_encoding_table(n_seq, d_hidn):
    def cal_angle(position, i_hidn):
        return position / np.power(10000, 2 * (i_hidn // 2)) / d_hidn

    def get_posi_anlge_vec(position):
        return[cal_angle(position, i_hidn) for i_hidn in range(d_hidn)] # 히든사이즈 별 angle 값을 구하기 위해 for문 사용
        

    # 각 position에 대해서 dim 마다 angle 값을 구함
    sinusoid_table = np.array([get_posi_anlge_vec(i_seq) for i_seq in range(n_seq)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])


    return sinusoid_table


#  시퀀스 내 각 위치에 대한 고유한 임베딩 테이블이 반환됩니다.
#  이러한 위치 임베딩은 Transformer와 같은 모델에서 입력 토큰의 위치 정보를 캡처하는 데 사용

In [26]:
n_seq = 64
pos_encoding = get_sinusoid_encoding_table(n_seq, d_hidn)
print(pos_encoding.shape)


# embedding 그림 출력
# plt.pcolormesh(pos_encoding, cmap='RdBu')
# plt.xlabel('Depth')
# plt.xlim((0, d_hidn))
# plt.ylabel('Position')
# plt.colorbar()
# plt.show()

(64, 128)


C:\Users\chunjae\AppData\Local\Temp\ipykernel_22004\472403830.py:4: RuntimeWarning: invalid value encountered in scalar divide
  return position / np.power(10000, 2 * (i_hidn // 2)) / d_hidn
C:\Users\chunjae\AppData\Local\Temp\ipykernel_22004\472403830.py:4: RuntimeWarning: divide by zero encountered in scalar divide
  return position / np.power(10000, 2 * (i_hidn // 2)) / d_hidn
C:\Users\chunjae\AppData\Local\Temp\ipykernel_22004\472403830.py:12: RuntimeWarning: invalid value encountered in sin
  sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:,0::2])
C:\Users\chunjae\AppData\Local\Temp\ipykernel_22004\472403830.py:13: RuntimeWarning: invalid value encountered in cos
  sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])


In [31]:
# position embedding 구성
pos_encoding = torch.FloatTensor(pos_encoding)

# embedding layer 생성
nn_pos = nn.Embedding.from_pretrained(pos_encoding, freeze=True) # freeze 학습을 하지 않겠다
nn_pos

# 각각의 input 값을 구하고, input 값의 포지션에 대한 임베딩을 가져오면 됨 

# input: [2, 8]
positions = torch.arange(inputs.size(1), device=inputs.device, dtype=inputs.dtype).expand(inputs.size(1)).contiguous() + 1 #0~7 에 해당하는 input 값
# 익스팬드는 차원을 확장하는데 2 바이 8
# 8에 해당하는 내용을 2개로 확장한다
# torch.arange() 함수를 사용하여 입력 시퀀스의 길이(inputs.size(1))만큼의 정수 시퀀스를 생성
# 이 시퀀스는 0부터 시작하여 입력 시퀀스의 길이 - 1까지의 값을 가집니다 0~7
# 따라서, 생성된 정수 시퀀스를 입력 시퀀스의 길이와 동일하게 확장합니다. + 1 이를 통해 입력 시퀀스의 길이만큼의 값이 반복되는 시퀀스를 생성
#.contiguous(): 확장된 시퀀스를 연속된 메모리 공간에 배치합니다. 
# 이는 메모리 공간을 연속적으로 사용하여 효율적인 연산을 수행하기 위해 필요



# 임베딩 백터를 만들어놓음
# 인풋값에 적용할 것
# 포지션 인풋은 0이냐 1이냐 3이냐 인풋값의 위치 정보
# 0번째에 있는 임베딩 벡터 값을 가져오기 위해
# 포지션 인풋값 = 위치 정보

# position masking 인풋값이 있느냐 없느냐를 찾음
pos_mask = inputs.eq(0)
pos_mask = pos_mask.unsqueeze(0).unsqueeze(-1)  # 2D 텐서로 변경
pos_mask = pos_mask.expand_as(positions)  # inputs와 positions의 모양을 맞춤
positions.masked_fill_(pos_mask, 0)  # padding 값을 마스킹
pos_embs = nn_pos(positions)

print(inputs)
print(positions)
print(pos_embs.size())


RuntimeError: expand(CPUBoolType{[1, 2, 8, 1]}, size=[8]): the number of sizes provided (1) must be greater or equal to the number of dimensions in the tensor (4)

In [ ]:
# 초기 input 값 구성
input_sums = input_embs + pos_embs
print(input_sums.size())

#### 6. Scale Dot Product Attention

##### Input

In [ ]:
# input 입력 값을 만드는 과정
Q = input_sums
K = input_sums
V = input_sums

##### Q * K-transpose

##### Scale

In [ ]:
# scaled dot

##### Mask (Opt.)

In [ ]:
# masking

##### Softmax

In [ ]:
# softmax 적용

##### atten_prov * V

##### Implementation Class

In [ ]:
""" scale dot product attention """
class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_head):
        super().__init__()
    
    def forward(self, Q, K, V, attn_mask):

        return context, attn_prob

#### 7. Multi-Head Attention

##### Input

In [ ]:
Q = input_sums
K = input_sums
V = input_sums
# masking 만들기 

##### Multi Head Q, K, V

In [ ]:
# 멀티 헤드 수에 맞게 linear 구성

In [ ]:
# 멀티 헤드 수에 맞게 변경 -> Q, K, V 모두

##### Multi Head Attention Mask

In [ ]:
# Mask도 변경

##### Attention

##### Concat

##### Linear

##### Implementation Class

In [ ]:
""" multi head attention """
class MultiHeadAttention(nn.Module):
    def __init__(self, d_hidn, n_head, d_head):
        super().__init__()
        # self 인자
        # linear, sacled_dot_attn, linear
    
    def forward(self, Q, K, V, attn_mask):
        batch_size = Q.size(0)
        # q_s: (bs, n_head, n_q_seq, d_head)

        # k_s: (bs, n_head, n_k_seq, d_head)

        # v_s: (bs, n_head, n_v_seq, d_head)

        # mask
        # (bs, n_head, n_q_seq, n_k_seq)

        # scaled dot
        # (bs, n_head, n_q_seq, d_head), (bs, n_head, n_q_seq, n_k_seq)

        # concat
        # (bs, n_q_seq, h_head * d_head)

        # linear
        # (bs, n_q_seq, d_hidn)

        # (bs, n_q_seq, d_hidn), (bs, n_head, n_q_seq, n_k_seq)
        return output, attn_prob

#### 8. Masked Multi Head Attention

In [ ]:
""" attention decoder mask """
def get_attn_decoder_mask(seq):
    return subsequent_mask


Q = input_sums
K = input_sums
V = input_sums

# attn_pad_mask : 기존 input에 대한 pad mask

# attn_dec_mask : 현재 단어에서 이전 단어만 보겠다는 attention mask

# attn_mask : attn_pad_mask + attn_dec_mask

batch_size = Q.size(0)
n_head = 2

#### 9. Feed Forward

##### f1 (Linear)

In [ ]:
# Linear : Conv1d로 활용
# (bs, d_hidn * 4, n_seq)

##### Activation (relu or gelu)

![](https://raw.githubusercontent.com/paul-hyun/paul-hyun.github.io/master/assets/2019-12-19/activation.png)

In [ ]:
# active = F.gelu

##### f3 (Linear)

In [ ]:
# Linear : Conv1d로 활용

##### Implementation Class

In [ ]:
""" feed forward """
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, d_hidn):
        super().__init__()

    def forward(self, inputs):
        # f1 output: (bs, d_ff, n_seq)

        # f2 output: (bs, n_seq, d_hidn)

        # (bs, n_seq, d_hidn)
        return output